In [1]:
import pandas as pd 
import requests as re
from datetime import datetime
import datetime
import pyodbc

# Essa função recebe uma string no tipo Hostname: HA010R01B13 <br> e retorna apenas o nome do robo. 
def getName(data):
    index= 0
    indNome_fim = 0
    indNome_flag_final = 0
    for y in data:
        if y == '<'  and indNome_flag_final == 0:
            indNome_fim = index
            indNome_flag_final = 1
        index = index + 1
    return data[1:indNome_fim]
# Retorna o valor do registrador da pagina html robo.  
def GetRegValue(html , reg):      
        posReg = html.find(reg)
        html = html[posReg:posReg+39] 
        index = 0
        indComment_inicio  = 0
        indComment_flag_inicio = 0
        indValue_inicio = 0
        indValue_inicio_flag = 0           
 ##Pega os indexadores dos valores desejados para separar a linha em comentario valor e numero do registrador. 
        for x in html :
                if x == "'" and indComment_flag_inicio == 0 :
                        indComment_inicio = index
                        indComment_flag_inicio = 1
                        break                     
                if x == "=" and indValue_inicio_flag == 0:
                        indValue_inicio = index + 2
                        indValue_inicio_flag = 1
                index = index + 1
        #Definindo Valores.   
        valueReg =float(html[indValue_inicio:indComment_inicio-2])
        return valueReg
# Retorna o comentario do registrador
def get_Comentario(data):
    index = 0
    indComment_inicio  = 0
    indComment_flag_inicio = 0
    indComment_fim = 0
    indComment_flag_final = 0
        
    ##Pega os indexadores dos valores desejados para separar a linha em comentario valor e numero do registrador. 
    for x in data:
        if x == "'" and indComment_flag_inicio == 0 :
                indComment_inicio = index + 1
                indComment_flag_inicio = 1  
        elif x == "'" and indComment_flag_inicio == 1 and indComment_flag_final == 0 :
                indComment_fim = index 
                indComment_flag_final = 1             
        index += 1
    return data[indComment_inicio:indComment_fim]
# Retorna o numero do registrador de acordo com o seu comentario
def getReg_Num(data):
    index = 0
    indReg_inicio = 0
    indReg_fim = 0
    indReg_flag_inicio = 0 
    indReg_flag_final = 0
        
    for x in data:     
        if x == "[" and indReg_flag_inicio == 0 :
                indReg_inicio = index + 1
                indReg_flag_inicio = 1
                
        if x == "]" and indReg_flag_inicio == 1 and indReg_flag_final == 0  :
                indReg_fim = index
                indReg_flag_final = 1
        index += 1
        #Definindo Valores.   
         
    return data[indReg_inicio:indReg_fim]
# Retorna o valor do registrador de uma substring.
def getReg_Value(data):
    index = 0
    indValue_inicio = 0
    indValue_final = 0
    indValue_flag_inicio = 0 
    indValue_flag_final = 0
        
    for x in data:     
        if x == "=" and indValue_flag_inicio == 0 :
                indValue_inicio = index + 2
                indValue_flag_inicio = 1
                
        if x == "'" and indValue_flag_inicio == 1 and indValue_flag_final == 0  :
                indValue_final = index - 2
                indValue_flag_final = 1
                break
        index += 1
         
    return data[indValue_inicio:indValue_final]



In [13]:
error_counter = 0
currntinc_counter = 0
preset_counter = 0 

server = 'BRSCTWDCS2403BK\SQLEXPRESS01'
database = 'REG_TIPDRESS'
username = 'sa'
password = '1234'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID ='+username+';PWD='+password)
cursor = connection.cursor()

IPS = pd.read_sql_query("SELECT* FROM TB_ROBOTS_MONITORED",connection)

# Limpar os dados da tabela para atualizar
#cursor.execute('DELETE FROM TB_REG_VALUES')
#cursor.commit()
# Lista para buscar entre os diferentes ranges de registradores utilizados:
TDRegList = [['[470]', '[471]', '[472]', '[474]','[475]'],
            ['[490]', '[491]', '[492]', '[494]','[495]'],
            ['[131]', '[132]', '[133]', '[134]','[135]']]

CURRNTRegList = ['[79]','[82]','[83]', '[92]']

# Percorrendo as linhas dos Endereços.
for i in range(304,312):
    if IPS.iloc[i, 2] == '470':
        selector = 0
    elif IPS.iloc[i, 2] == '490':
        selector = 1
    elif IPS.iloc[i, 2] == '131':
        selector = 2
    else:
        selector = 3

    try:
        response = re.get('http://' + IPS.iloc[i, 1] + '/MD/NUMREG.VA')
    except:
        print(IPS.iloc[i, 1], 'Get Error')
        error_counter += 1 
    else:
        html = response.text
        posName = html.find("Hostname")
        robot_name = getName(html[posName+9:posName+25])
        robot_date = datetime.datetime.now().strftime('%d/%m/%Y')
        robot_time = datetime.datetime.now().strftime('%H:%M:%S')
        if selector != 3:
            wear_mem = GetRegValue (html , TDRegList[selector][0])
            result = GetRegValue (html , TDRegList[selector][1])
            preset = GetRegValue (html , TDRegList[selector][2])
            retrypreset = GetRegValue(html , TDRegList[selector][3])
            dresscount = GetRegValue(html, TDRegList[selector][4])
            currntinc_counter += 1
            preset_counter += 1
        
        if selector == 3:
            wear_mem = 0
            result = 0
            preset = 0
            retrypreset = 0
            dresscount = 0
            currntinc_counter += 1

        currentinc = GetRegValue (html , CURRNTRegList[0])
        currentfree = GetRegValue(html , CURRNTRegList[1])
        currentload = GetRegValue(html , CURRNTRegList[2])
        currentmax = GetRegValue (html,  CURRNTRegList[3])
            
        if selector != 2:
            SGcurrent = GetRegValue(html, '[488]')
        else:
            SGcurrent = Null   
                
        print(robot_name,IPS.iloc[i, 1], SGcurrent )
        insert = f"""INSERT INTO TB_REG_VALUES(robot_date, robot_time, robot_name,WEAR_MEM, RESULT, PRESET,RETRY_PRESET,DRESS_COUNT,
        CURRNT_INC, CURRNT_FREE, CURRNT_LOAD, CURRNT_MAX,
        SG_CURRNT)
            VALUES('{robot_date}','{robot_time}','{robot_name}',{wear_mem},{result},{preset},{retrypreset},{dresscount}, 
                    {currentinc},{currentfree},{currentload},{currentmax},
                    {SGcurrent})"""
        
        insert_history = f"""INSERT INTO TB_REG_VALUES_HISTORY(robot_date, robot_time, robot_name,WEAR_MEM, RESULT, PRESET,RETRY_PRESET,DRESS_COUNT,CURRNT_INC, CURRNT_FREE, CURRNT_LOAD, CURRNT_MAX,
        SG_CURRNT)
            VALUES('{robot_date}','{robot_time}','{robot_name}',{wear_mem},{result},{preset},{retrypreset},{dresscount}, 
                    {currentinc},{currentfree},{currentload},{currentmax},
                    {SGcurrent})"""

        #cursor.execute(insert_history)
        #cursor.commit()
        cursor.execute(insert)
        cursor.commit()

C:\Users\tz1zjg\AppData\Local\Temp\ipykernel_24984\3411832441.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  IPS = pd.read_sql_query("SELECT* FROM TB_ROBOTS_MONITORED",connection)


LA070R03 124.11.141.68 None


ProgrammingError: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Nome de coluna 'None' inválido. (207) (SQLExecDirectW)")

In [16]:
datetime.datetime.today().weekday()

4